In [1]:
import json

from qiskit import transpile
from qiskit.circuit.library import QAOAAnsatz, CXGate
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.passes.routing.commuting_2q_gate_routing import SwapStrategy

from qiskit_ibm_provider import IBMProvider

from qopt_best_practices.swap_strategies import create_qaoa_swap_circuit
from qopt_best_practices.utils import load_edge_coloring

In [2]:
with open("data/graph_hardware_native_127nodes_001.json", "r") as fin:
    data = json.load(fin)

cost_operator = SparsePauliOp.from_list(data["paulis"])

In [3]:
provider = IBMProvider(instance="quantum-demonstrations/main/locc")
backend = provider.get_backend("ibm_kyiv")

## Circuit creation

The problem is hardware native. Therefor, if we transpile its QAOA circuit to a 127 qubit backend with CNOT gates and a heavy-hexagonal coupling map then we expect to have (i) 288 CX gates per QAOA layer because a 127 qubit backend has 144 edges in its coupling map and each `RZZGate` requires two CNOT gates, and (ii) a CNOT depth of 6 because the heavy-hex coupling map has a maximum node degree of three. We will first construct a circuit with the standard Qiskit transpilation flow and show that it is sub-optimal.

In [6]:
p = 2
qaoa_circuit = QAOAAnsatz(cost_operator, reps=p)
qaoa_circuit = transpile(qaoa_circuit, backend, optimization_level=3)

In [7]:
print("Number of two-qubit gates:", qaoa_circuit.count_ops()["cx"])
print("CNOT gate depth:", qaoa_circuit.depth(filter_function=lambda x: x.operation == CXGate()))

Number of two-qubit gates: 576
CNOT gate depth: 80


We can see above that the Qiskit transpiler has produced a hardware native circuit, because the number of two-qubit gates is the minimum number we need, i.e., `p * 2 * 144`. However, something went wrong in the circuit depth. We have a CNOT gate depth of 80 when we were expecting `2 * p * 3`. This shows that the gates have been staggereg instead of placed in parallel. To fix this we will transpile with an empty swap network. We chose an empty network because we do not need swaps but we still need the ability of this transpiler pass to place commuting two-qubit gates in parallel.

In [8]:
edge_coloring = load_edge_coloring()
no_swaps = SwapStrategy(backend.coupling_map, ())
qaoa_circuit = transpile(create_qaoa_swap_circuit(cost_operator, no_swaps, qaoa_layers=p, edge_coloring=edge_coloring), backend)

In [9]:
print("Number of two-qubit gates:", qaoa_circuit.count_ops()["cx"])
print("CNOT gate depth:", qaoa_circuit.depth(filter_function=lambda x: x.operation == CXGate()))

Number of two-qubit gates: 576
CNOT gate depth: 12


We now have the expected circuit depth. Here, `2 * p * 3`.

## Dynamical decoupling

The hardware native circuit will have plenty of idling qubits. Indeed, on a heavy-hexagonal coupling map if a CX gate is applied to an edge then we are guarenteed to have at least one idling qubit given that on any edge one node has degree two and one node has degree three. In the following we will insert dynamical decoupling gate sequences to protect the idling qubits.

In [10]:
from qopt_best_practices.transpilation import dd_pass_manager

In [12]:
dd_xx = dd_pass_manager(backend, "XX")

In [14]:
xx_circuit = dd_xx.run(qaoa_circuit)

In [15]:
qaoa_circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

In [18]:
from denoising.light_cone_qaoa import LightConeQAOA
from qopt_best_practices.utils import build_max_cut_graph

In [20]:
graph = build_max_cut_graph(data["paulis"])

In [24]:
import networkx as nx

In [27]:
nx.from_edgelist([(0, 1), (1, 2)])

In [22]:
for edge in graph.edges():
    print(edge)

(0, 1)
(0, 14)
(1, 2)
(2, 3)
(3, 4)
(4, 15)
(4, 5)
(15, 22)
(5, 6)
(6, 7)
(7, 8)
(8, 9)
(8, 16)
(9, 10)
(10, 11)
(11, 12)
(12, 17)
(12, 13)
(17, 30)
(14, 18)
(16, 26)
(30, 29)
(30, 31)
(18, 19)
(19, 20)
(20, 33)
(20, 21)
(33, 39)
(21, 22)
(22, 23)
(23, 24)
(24, 25)
(24, 34)
(25, 26)
(26, 27)
(27, 28)
(28, 29)
(28, 35)
(31, 32)
(32, 36)
(36, 51)
(39, 38)
(39, 40)
(34, 43)
(35, 47)
(47, 46)
(47, 48)
(51, 50)
(37, 38)
(37, 52)
(40, 41)
(41, 42)
(41, 53)
(42, 43)
(43, 44)
(44, 45)
(45, 46)
(45, 54)
(48, 49)
(49, 50)
(49, 55)
(52, 56)
(56, 57)
(53, 60)
(54, 64)
(64, 63)
(64, 65)
(55, 68)
(68, 67)
(68, 69)
(57, 58)
(58, 59)
(58, 71)
(59, 60)
(71, 77)
(60, 61)
(61, 62)
(62, 63)
(62, 72)
(65, 66)
(66, 73)
(66, 67)
(73, 85)
(69, 70)
(70, 74)
(74, 89)
(72, 81)
(81, 80)
(81, 82)
(85, 84)
(85, 86)
(89, 88)
(75, 76)
(75, 90)
(76, 77)
(77, 78)
(78, 79)
(79, 80)
(79, 91)
(82, 83)
(83, 92)
(83, 84)
(92, 102)
(86, 87)
(87, 88)
(87, 93)
(90, 94)
(91, 98)
(102, 101)
(102, 103)
(93, 106)
(106, 105)
(106, 